In [1]:
import matplotlib.pyplot as plt
from utils import Metric
import numpy as np
import os
import data
import sys

sys.argv = ["notebook", "--datadir", "/mnt/sda1/arash/data/solar_flare",
            "--logdir", "log",
            "--resultfilename", "feature_selection",
            "--resultdir", "experiments_plot5",
            "--gpu", "1",
            "--runs", "5",
            "--experiment", "feature_selection",
            "--depth", "2,4,8",
            "--kernelsize", "9,9,9",
            "--nan", "0",
            "--batch", "64",
            "--norm", "scale",
            "--poolingsize", "2",
            "--poolingstrat", "max",
            "--hidden", "128,32",
            "--n", "6500,1000",
            "--valp", "0.5",
            "--importance", "0.5,0.5",
            "--datadrop", "0.0",
            "--layerdrop", "0.3",
            "--seed", "43",
            "--earlystop", "40",
            "--stop", "200",
            "--verbose", "3",
            "--splitreport", "split_best.csv",
            "--modelreport", "model_best.csv",
            "--configreport", "config_best.csv",
            "--aug", "--ndbsr", "--smote", "--cache"]
import utils

args = utils.arg_parse()
args.saliency = True

In [2]:
from data import Data, data_columns
from reporter import Reporter

data = Data(args, verbose=False)
reporter = Reporter()

Reading all files df ...
/mnt/sda1/arash/data/solar_flare/full_data_X_1_25.npy
Reading all files np ...
(331185, 24, 60)
(11260011,)


In [3]:
saliency = np.load("saliency.npy")

In [4]:
saliency_sum = np.sum(saliency, axis=1)

In [5]:
feature_names = data_columns(args)[1:25]

In [6]:
feature_names, saliency_sum

(Index(['TOTUSJH', 'TOTBSQ', 'TOTPOT', 'TOTUSJZ', 'ABSNJZH', 'SAVNCPP',
        'USFLUX', 'TOTFZ', 'MEANPOT', 'EPSZ', 'MEANSHR', 'SHRGT45', 'MEANGAM',
        'MEANGBT', 'MEANGBZ', 'MEANGBH', 'MEANJZH', 'TOTFY', 'MEANJZD',
        'MEANALP', 'TOTFX', 'EPSY', 'EPSX', 'R_VALUE'],
       dtype='object'),
 array([528.69708019, 164.64273374, 239.58871714, 271.22483922,
        433.34870452, 358.35988629, 199.70431151,  45.69827707,
        217.70550826,  59.04669959,  38.43489543,  77.54180944,
         36.81264113,  49.65307657,  68.2387864 , 153.10216279,
         39.05107782,  44.65792978,  69.34814983,  46.30592577,
         55.07855463,  61.74069276,  50.98924402, 168.14655612]))

In [7]:
ordering = sorted(range(len(saliency_sum)), key=lambda i:saliency_sum[i], reverse=True)

In [8]:
print([feature_names[i] for i in ordering])

['TOTUSJH', 'ABSNJZH', 'SAVNCPP', 'TOTUSJZ', 'TOTPOT', 'MEANPOT', 'USFLUX', 'R_VALUE', 'TOTBSQ', 'MEANGBH', 'SHRGT45', 'MEANJZD', 'MEANGBZ', 'EPSY', 'EPSZ', 'TOTFX', 'EPSX', 'MEANGBT', 'MEANALP', 'TOTFZ', 'TOTFY', 'MEANJZH', 'MEANSHR', 'MEANGAM']


In [9]:
print(ordering)

[0, 4, 5, 3, 2, 8, 6, 23, 1, 15, 11, 18, 14, 21, 9, 20, 22, 13, 19, 7, 17, 16, 10, 12]


In [12]:
args.ordering = ordering
args.length = 6

In [13]:
args.test_part = 5
train, val, test = data.dataholders(args, *data.numpy_datasets(args, 0))

0
There are (3249, 501) instances.
There are (2787, 501) instances, 462 removed
in smote


In [14]:
def select_indices(train, val, test, args):
    selected_indices = args.ordering[:args.length]
    print(train.dataset.X.shape)
    print(val[0].X.shape)
    print(test[0].X.shape)
    print(selected_indices)
    train.dataset.X = train.dataset.X[:, selected_indices, :]

    val[0] = utils.DataPair(val[0].X[:, selected_indices, :], val[0].y)

    for i in range(len(test)):
        test[i] = utils.DataPair(test[i].X[:, selected_indices, :], test[i].y)

    return train, val, test

In [15]:
train_new, val_new, test_new = select_indices(train, val, test, args)

torch.Size([5574, 24, 60])
torch.Size([3750, 24, 60])
torch.Size([64, 24, 60])
[0, 4, 5, 3, 2, 8]


In [16]:
val[0].X.shape

torch.Size([3750, 6, 60])

In [17]:
train.dataset.X.shape

torch.Size([5574, 6, 60])

In [18]:
import train as t

In [20]:
args.run_no = 0
args.n_features = 6
args.verbose = 5
t.train(args, data, reporter)

There are (3249, 501) instances.
There are (2787, 501) instances, 462 removed
in smote
torch.Size([5574, 24, 60])
torch.Size([3750, 24, 60])
torch.Size([64, 24, 60])
[0, 4, 5, 3, 2, 8]


/home/arash/.virtualenvs/solar_flare_cnn/lib/python3.11/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


run no. 0, test 5:75113, duration 2119.1 ms
Total number of parameters: 12000
run no. 0, test 5, best val run: Metric(tss: 81.70, f1:  67.91, cm: [[2832, 419], [27, 472]])
run no. 0, test 5, test run    : Metric(tss: 84.55, f1:  21.59, cm: [[67462, 6661], [64, 926]])


ValueError: operands could not be broadcast together with shapes (24,60) (6,60) (24,60) 